## Creando Kafka Consumer

In [2]:
from kafka import KafkaConsumer

# Configuración básica del consumidor
bootstrap_servers = 'host.docker.internal:9092'
group_id = 'test_3'
topic = "tpc-test"

# Instanciar el consumidor
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=bootstrap_servers,
    group_id=group_id,
    auto_offset_reset='earliest'  # Esto hará que el consumidor lea desde el inicio del tópico si no hay un offset previamente guardado.
)

try:
    for message in consumer:
        print(message.value)  # Mostrar el valor del mensaje

except KeyboardInterrupt:
    print("Detenido por el usuario")
finally:
    consumer.close()



b'{"productId": 1, "product_name": "ABC"}'
b'{"productId": 2, "product_name": "DEF"}'
b'{"productId": 3, "product_name": "GHI"}'
Detenido por el usuario


In [5]:
from kafka import KafkaConsumer

# Configuración básica del consumidor
bootstrap_servers = 'host.docker.internal:9092'
group_id = 'test_3'
topic = "tpc-test"


consumer = KafkaConsumer(
    topic,
    bootstrap_servers=bootstrap_servers,
    group_id=group_id,
    auto_offset_reset='earliest',  # Leer desde el inicio del tópico si no hay un offset previamente guardado.
    value_deserializer=lambda x: x.decode('utf-8'),  # Deserializar los mensajes como UTF-8
    enable_auto_commit=False
)


The drawback is that while commitSync() will retry the commit until it either succeeds or encounters a nonretriable failure, commitAsync() will not retry

In [4]:
try:
    for message in consumer:
        # Mostrar información detallada del mensaje
        print(
            f"topic = {message.topic}, partition = {message.partition}, offset = {message.offset}, "
            f"key = {message.key}, value = {message.value}"
        )

        # Realizar commit síncrono del offset después de procesar el mensaje
        consumer.commit()

except KeyboardInterrupt:
    print("Detenido por el usuario")
finally:
    consumer.close()  # Asegurarse de cerrar el consumidor al final

topic = tpc-test, partition = 0, offset = 0, key = None, value = {"productId": 1, "product_name": "ABD"}
topic = tpc-test, partition = 0, offset = 1, key = None, value = {"productId": 2, "product_name": "EFG"}
topic = tpc-test, partition = 0, offset = 2, key = None, value = {"productId": 3, "product_name": "HIJ"}
Detenido por el usuario


In [6]:
try:
    for message in consumer:
        # Mostrar información detallada del mensaje
        print(
            f"topic = {message.topic}, partition = {message.partition}, offset = {message.offset}, "
            f"key = {message.key}, value = {message.value}"
        )

        # Realizar commit asíncrono del offset después de procesar el mensaje
        consumer.commit_async()

except KeyboardInterrupt:
    print("Detenido por el usuario")
finally:
    consumer.close()  

topic = tpc-test, partition = 0, offset = 0, key = None, value = {"productId": 1, "product_name": "ABD"}
topic = tpc-test, partition = 0, offset = 1, key = None, value = {"productId": 2, "product_name": "EFG"}
topic = tpc-test, partition = 0, offset = 2, key = None, value = {"productId": 3, "product_name": "HIJ"}
Detenido por el usuario


## Commit Specific offset


En la biblioteca kafka-python, el resultado de consumer.poll() es un objeto ConsumerRecords, que es fundamentalmente un diccionario cuyas claves son objetos TopicPartition y cuyos valores son listas de objetos ConsumerRecord

In [10]:
from kafka import KafkaConsumer, TopicPartition, OffsetAndMetadata

# Configuración básica del consumidor
bootstrap_servers = 'host.docker.internal:9092'
group_id = 'test_2'
topic = "tpc-test-cons"
poll_timeout = 0.1  # 100 milliseconds

# Instanciar el consumidor
consumer = KafkaConsumer(
    topic,
    bootstrap_servers=bootstrap_servers,
    group_id=group_id,
    auto_offset_reset='earliest',  # Leer desde el inicio del tópico si no hay un offset previamente guardado.
    value_deserializer=lambda x: x.decode('utf-8'),  # Deserializar los mensajes como UTF-8
    enable_auto_commit=False  # Desactivar el auto commit
)

current_offsets = {}  # Diccionario para mantener los offsets actuales
count = 0

try:
    while True:
        records = consumer.poll(timeout_ms=poll_timeout*1000)  # Convertir a milisegundos

        for record_batch in records.values():
            for message in record_batch:
                # Actualizar el offset en el diccionario
                tp = TopicPartition(message.topic, message.partition)
                current_offsets[tp] = OffsetAndMetadata(message.offset + 1, "no metadata")
                
                # Realizar commit asíncrono después de procesar cada 1000 mensajes
                count += 1
                if count % 1000 == 0:
                    print(
                            f"topic = {message.topic}, partition = {message.partition}, offset = {message.offset}, "
                            f"key = {message.key}, value = {message.value}"
                        )
                    consumer.commit_async(offsets=current_offsets)
except KeyboardInterrupt:
    print("Detenido por el usuario")
finally:
    consumer.close()  # Asegurarse de cerrar el consumidor al final


topic = tpc-test-cons, partition = 0, offset = 999, key = None, value = {"ID": 999, "name": "4Ym40DkUCWjOPZc"}
topic = tpc-test-cons, partition = 0, offset = 1999, key = None, value = {"ID": 999, "name": "cOmRD11MSbc4Zyl"}
topic = tpc-test-cons, partition = 0, offset = 2999, key = None, value = {"ID": 999, "name": "9ca6ASb9zoknuI8"}
Detenido por el usuario


In [ ]:
from kafka import KafkaConsumer, TopicPartition, OffsetAndMetadata

bootstrap_servers = 'your_bootstrap_servers'
group_id = 'your_group_id'
topics = ['your_topic']
poll_timeout = 0.1  # equivalente a Duration.ofMillis(100)

current_offsets = {}

# Definir los callbacks de rebalance
def on_partitions_assigned(consumer, partitions):
    # Aquí puedes manejar lógica para cuando se asignan particiones
    pass

def on_partitions_revoked(consumer, partitions):
    print(f"Lost partitions in rebalance. Committing current offsets: {current_offsets}")
    consumer.commit(offsets=current_offsets)

# Configurar e iniciar el consumidor
consumer = KafkaConsumer(
    bootstrap_servers=bootstrap_servers,
    group_id=group_id,
    enable_auto_commit=False,
    auto_offset_reset='earliest'
)

consumer.subscribe(topics, on_partitions_assigned=on_partitions_assigned, on_partitions_revoked=on_partitions_revoked)

try:
    while True:
        records = consumer.poll(timeout_ms=poll_timeout*1000)  # Convertir a milisegundos
        for message in records.values():
            for record in message:
                print(
                    f"topic = {record.topic}, partition = {record.partition}, offset = {record.offset}, "
                    f"key = {record.key}, value = {record.value}"
                )
                tp = TopicPartition(record.topic, record.partition)
                current_offsets[tp] = OffsetAndMetadata(record.offset+1, "")
        consumer.commit_async(offsets=current_offsets)

except KeyboardInterrupt:  # Manejar interrupción manual
    print("Detenido por el usuario")

except Exception as e:
    print(f"Error inesperado: {e}")

finally:
    try:
        consumer.commit(offsets=current_offsets)
    finally:
        consumer.close()
        print("Closed consumer and we are done")
